<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

使用RestAPI获取POI（只用types获取）
https://restapi.amap.com/v3/place/text?types=150500&city=2101&output=json&offset=100&page=1&key=0c5ccf69838f9de4ea0f118271c0f34f&extensions=all

使用
https://www.amap.com/service/poiInfo?query_type=TQUERY&city=2101&keywords=地铁1号线

In [1]:
import requests
from pyecharts import options as opts
from pyecharts.charts import *
from pyecharts.commons.utils import JsCode

subways = ['地铁1号线', '地铁2号线', '地铁9号线']

# headers信息 ,referer暂时不管        
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.146 Safari/537.36',
           'Referer': 'https://sz.lianjia.com/ershoufang/'}

map_data = []
for sbw in subways:
    temp_list = []
    url = 'https://www.amap.com/service/poiInfo?query_type=TQUERY&city=2101&keywords={}'.format(sbw)
    data = requests.get(url, headers=headers).json()
    # print(data)
    for item in data['data']['busline_list'][0]['stations']:
        x, y = item['xy_coords'].split(';')
        temp_list.append([float(x), float(y)])
    map_data.append(temp_list)
print(map_data)

KeyError: 'data'

In [91]:
bmap = BMap()
bmap.add_schema(
    # 需要申请一个AK
    baidu_ak='GNCdXmuXlZjj6MdsZG1lGIbkM8NF2qzs',
    # 地图缩放比例
    zoom=12, 
    # 显示地图中心坐标点
    center=[123.44,41.79],
    is_roam=True,
    map_style={
            "styleJson": [
                {
                    "featureType": "water",
                    "elementType": "all",
                    "stylers": {"color": "#031628"},
                },
                {
                    "featureType": "land",
                    "elementType": "geometry",
                    "stylers": {"color": "#000102"},
                },
                {
                    "featureType": "highway",
                    "elementType": "all",
                    "stylers": {"visibility": "off"},
                },
                {
                    "featureType": "arterial",
                    "elementType": "geometry.fill",
                    "stylers": {"color": "#000000"},
                },
                {
                    "featureType": "arterial",
                    "elementType": "geometry.stroke",
                    "stylers": {"color": "#0b3d51"},
                },
                {
                    "featureType": "local",
                    "elementType": "geometry",
                    "stylers": {"color": "#000000"},
                },
                {
                    "featureType": "railway",
                    "elementType": "geometry.fill",
                    "stylers": {"color": "#000000"},
                },
                {
                    "featureType": "railway",
                    "elementType": "geometry.stroke",
                    "stylers": {"color": "#08304b"},
                },
                {
                    "featureType": "subway",
                    "elementType": "geometry",
                    "stylers": {"visibility": "off"},
                },
                {
                    "featureType": "building",
                    "elementType": "geometry.fill",
                    "stylers": {"color": "#000000"},
                },
                {
                    "featureType": "all",
                    "elementType": "labels.text.fill",
                    "stylers": {"color": "#d1dede"},
                },
                {
                    "featureType": "all",
                    "elementType": "labels.text.stroke",
                    "stylers": {"color": "#000000"},
                },
                {
                    "featureType": "building",
                    "elementType": "geometry",
                    "stylers": {"color": "#022338"},
                },
                {
                    "featureType": "green",
                    "elementType": "geometry",
                    "stylers": {"color": "#062032"},
                },
                {
                    "featureType": "boundary",
                    "elementType": "all",
                    "stylers": {"color": "#465b6c"},
                },
                {
                    "featureType": "manmade",
                    "elementType": "all",
                    "stylers": {"color": "#022338"},
                },
                {
                    "featureType": "label",
                    "elementType": "all",
                    "stylers": {"visibility": "off"},
                },
            ]
        },)

# 添加数据
colors = ['#e62739','#7bc8a4','#4cc3d9']
bmap.set_global_opts(legend_opts=opts.LegendOpts(
                    textstyle_opts=opts.TextStyleOpts(color='#d1dede'),pos_left=10,pos_top=10))
for sub_data in map_data:
    bmap.add(subways[map_data.index(sub_data)],
            [sub_data],
            type_='lines',
            is_polyline=True,
            linestyle_opts=opts.LineStyleOpts(opacity=0.2, width=1,color=colors[map_data.index(sub_data)]),
            progressive=200,
            progressive_threshold=500,)
         
bmap.render('charts/沈阳地铁.html')

'/Users/lhc/dev/repo/prac_py/charts/沈阳地铁.html'